In [ ]:
# 1) Convert files for Metatlas 2.0 (using mzCAT)
# 2) Transfer to NERSC (using mzCAT)
# 3) Make MetAtlas Groups for your files
# 4) Enter your internal standards into an Atlas
# 5) Extract data from each of your files for the internal standards
# 6) Export the results in a meaningful way

# If all goes according to plan that means on Thursday we will:
# 1) Build a small Atlas of your favorite molecules that have been identified.
# 2) Extract data from each of your files for these molecules
# 3) Export the results in a meaningful way

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import sys,os,glob,csv
import numpy as np
from matplotlib import pylab as plt

from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import fcluster
from scipy.spatial.distance import squareform

sys.path.insert(0,'/global/project/projectdirs/metatlas/anaconda/lib/python2.7/site-packages' )
from metatlas import metatlas_objects as metob
from metatlas import h5_query

curr_ld_lib_path = ''

os.environ['LD_LIBRARY_PATH'] = curr_ld_lib_path + ':/project/projectdirs/openmsi/jupyterhub_libs/boost_1_55_0/lib' + ':/project/projectdirs/openmsi/jupyterhub_libs/lib'
# sys.path.remove('/anaconda/lib/python2.7/site-packages')
sys.path.append('/global/project/projectdirs/openmsi/jupyterhub_libs/anaconda/lib/python2.7/site-packages')
sys.path.insert(0,'/project/projectdirs/openmsi/projects/meta-iq/pactolus/pactolus' )

from rdkit import Chem
# from rdkit.Chem.rdMolDescriptors import ExactMolWt
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import AllChem
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.Chem import Draw
from rdkit import DataStructs
from rdkit.DataManip import Metric

import networkx as nx

In [89]:
atlases = metob.retrieve('Atlas',name='All%')
for a in atlases:
    print a.name
# atlases = metob.retrieve('Atlas',name='QExactive%')
myAtlas = atlases[0]
istds = []

for cid in myAtlas.compound_identifications:
    myreferences = cid.retrieve()
    c = myreferences.compound.retrieve()
    for r in myreferences.references:
        temp = r.retrieve()
        if 'mz' in dir(temp):
            print c.name, temp.mz, temp.mz_tolerance, temp.mz_tolerance_units
            mzmin = temp.mz - temp.mz*temp.mz_tolerance / 1e6
            mzmax = temp.mz + temp.mz*temp.mz_tolerance / 1e6
            istds.append({'name':c.name,'mzmin':mzmin,'mzmax':mzmax})
        if 'RTpeak' in dir(temp):
            print c.name, temp.RTpeak, temp.RTmin, temp.RTmax, temp.RTUnits



All Encompasing Internal Standards Positive Mode
3,6-Dihydroxy-4-methylpyridazine 127.05 20.0 ppm
d5-benzoic acid 128.075 20.0 ppm
4-(3,3-dimethyl-ureido)benzoic acid 209.092 20.0 ppm
9-anthracene carboxylic acid 223.075 20.0 ppm
13C-15N-L-phenylalanine 176.113 20.0 ppm
13C-glucose 187.091 20.0 ppm
d8-lysine 155.163 20.0 ppm
d4 lysine 151.138 20.0 ppm
ABMBA 229.981 20.0 ppm


In [3]:
# groups = metob.retrieve('Group')#,name='20150826_KZH_QExactive_Hilic_Avena_exudates_and_standards')
# for g in groups:
#     print g.name

In [76]:
# groups = metob.retrieve('Group',name='20150826_KZH_QExactive_Hilic_Avena_exudates_and_standards')
groups = metob.retrieve('Group',name='20151002_TLS_QExactive_Hilic_Crust Porewater')
myExperiment = groups[0]

for i,treatment_groups in enumerate(myExperiment.items):
    file_list = treatment_groups.retrieve()
    for j, files in enumerate(file_list.items):
        lcms_run = files.retrieve()
        print i,j,file_list.name,lcms_run.hdf5_file


0 0 10_18h /global/project/projectdirs/metatlas//raw_data/tls/151002_QE_porewater_pos/150930_HILIC_TS_pos_10E_18h.h5
0 1 10_18h /global/project/projectdirs/metatlas//raw_data/tls/151002_QE_porewater_neg/150930_HILIC_TS_neg_10D_18h.h5
0 2 10_18h /global/project/projectdirs/metatlas//raw_data/tls/151002_QE_porewater_pos/150930_HILIC_TS_pos_10D_18h.h5
0 3 10_18h /global/project/projectdirs/metatlas//raw_data/tls/151002_QE_porewater_pos/150930_HILIC_TS_pos_10C_18h.h5
0 4 10_18h /global/project/projectdirs/metatlas//raw_data/tls/151002_QE_porewater_neg/150930_HILIC_TS_neg_10E_18h.h5
0 5 10_18h /global/project/projectdirs/metatlas//raw_data/tls/151002_QE_porewater_pos/150930_HILIC_TS_pos_10B_18h.h5
0 6 10_18h /global/project/projectdirs/metatlas//raw_data/tls/151002_QE_porewater_neg/150930_HILIC_TS_neg_10C_18h.h5
0 7 10_18h /global/project/projectdirs/metatlas//raw_data/tls/151002_QE_porewater_neg/150930_HILIC_TS_neg_10B_18h.h5
1 0 11_18h /global/project/projectdirs/metatlas//raw_data/tls/15

In [7]:
from metatlas import h5_query as h5q
import tables


In [122]:
# print c.name, temp.mz, temp.mz_tolerance, temp.mz_tolerance_units
# print c.name, temp.RTpeak, temp.RTmin, temp.RTmax, temp.RTUnits
data = []
for treatment_groups in myExperiment.items:
    file_list = treatment_groups.retrieve()
    for i,files in enumerate(file_list.items):
        lcms_run = files.retrieve()
        print lcms_run
        if '_pos_' in lcms_run.hdf5_file.lower():
            h5file = tables.open_file(lcms_run.hdf5_file)
            for istd in istds[:4]:
                print istd
                rt,intensity = h5q.get_chromatogram(h5file,istd['mzmin']-0.01,istd['mzmax']+0.01,1,'positive')

                data.append((rt,intensity,istd['name'],file_list.name,lcms_run.hdf5_file))
#                 except:
#                     print "no data in ",lcms_run.hdf5_file    

#                 target = 132.1019 #leucine
#             target = 150.058
#             target = 136.061
#             target = 244.092776 #cytidine
#             mzmin = target - 15 * target / 1e6# 208.085 - 0.05
#             mzmax = target + 15 * target / 1e6#208.085 + 0.05
#             for cid in myAtlas.compound_identifications:
#                 myreferences = cid.retrieve()
#                 c = myreferences.compound.retrieve()

#                 for r in myreferences.references:
#                     temp = r.retrieve()
#                     if 'mz' in dir(temp):
#                         mzmin = temp.mz - temp.mz*10*temp.mz_tolerance / 1e6
#                         mzmax = temp.mz - temp.mz*10*temp.mz_tolerance / 1e6


150930_HILIC_TS_pos_10E_18h.mzML (a0580673c7884c298efdeda0a17d759d)
{'mzmax': 127.05254099999999, 'name': u'3,6-Dihydroxy-4-methylpyridazine', 'mzmin': 127.047459}
{'mzmax': 128.0775615, 'name': u'd5-benzoic acid', 'mzmin': 128.07243849999998}
{'mzmax': 209.09618184, 'name': u'4-(3,3-dimethyl-ureido)benzoic acid', 'mzmin': 209.08781816}
{'mzmax': 223.07946149999998, 'name': u'9-anthracene carboxylic acid', 'mzmin': 223.0705385}
150930_HILIC_TS_neg_10D_18h.mzML (61252f91684d4fa284b96cdc74d7f7d7)
150930_HILIC_TS_pos_10D_18h.mzML (f306eab1d0a24d8690ea4830f424c507)
{'mzmax': 127.05254099999999, 'name': u'3,6-Dihydroxy-4-methylpyridazine', 'mzmin': 127.047459}
{'mzmax': 128.0775615, 'name': u'd5-benzoic acid', 'mzmin': 128.07243849999998}
{'mzmax': 209.09618184, 'name': u'4-(3,3-dimethyl-ureido)benzoic acid', 'mzmin': 209.08781816}
{'mzmax': 223.07946149999998, 'name': u'9-anthracene carboxylic acid', 'mzmin': 223.0705385}
150930_HILIC_TS_pos_10C_18h.mzML (4bec3d322155456381f04be07efa0252)


In [125]:
len(data)/4

67

In [127]:
myCounter = 0
for i,d in enumerate(data):
    if len(d[0]) > 1:
        plt.subplot(67,4,myCounter+1)
        plt.plot(d[0],d[1])
        plt.title('%s\n%s'%(d[3],d[2]))
        plt.xlim(2,22)
        plt.ylim(0,7e8)
        myCounter = myCounter + 1
fig = plt.gcf()
fig.set_size_inches(22, 120)
fig.tight_layout()
plt.show()

In [70]:
# myFile = '/global/project/projectdirs/metatlas//raw_data/katezh/20150826_KZH_pHILIC_QExactive_ExudatesAndCassettes/20150826_pHILIC_POS_MSMS_ACN__Root_exudate_2_3x_R2_8ul.h5'
myFile = '/project/projectdirs/metatlas/raw_data/pandeer/20150305_angelo_MSMS/POS_5Dmsms.h5'
h5file = tables.open_file(myFile)



In [71]:
mz_bins = np.linspace(50,1000,1000)
data = h5q.get_heatmap(myFile,mz_bins,1,1)

In [75]:
print h5file

/project/projectdirs/metatlas/raw_data/pandeer/20150305_angelo_MSMS/POS_5Dmsms.h5 (File) ''
Last modif.: 'Fri Oct 16 00:31:15 2015'
Object Tree: 
/ (RootGroup) ''
/ms1_neg (Table(0,), shuffle, blosc(1)) ''
/ms1_pos (Table(1577,), shuffle, blosc(1)) ''
/ms2_neg (Table(0,), shuffle, blosc(1)) ''
/ms2_pos (Table(363,), shuffle, blosc(1)) ''



In [128]:
# import seaborn as sns; sns.set()
# plt.figure(figsize=(22,22))
# ax = sns.heatmap((data['arr']+1)**0.5,xticklabels=False,yticklabels=False,vmin=0,vmax=300)

In [129]:
%%javascript
var nb = IPython.notebook;
var kernel = IPython.notebook.kernel;
var command = "NOTEBOOK_FULL_PATH = '" + nb.base_url + nb.notebook_path + "'";
kernel.execute(command);

<IPython.core.display.Javascript object>

In [130]:
filename = os.path.basename(NOTEBOOK_FULL_PATH)
%system cp $filename /project/projectdirs/openmsi/www/
temp = '%s/%s'%('/project/projectdirs/openmsi/www',filename)
%system chmod 775 $temp
print 'http://nbviewer.ipython.org/url/portal.nersc.gov/project/openmsi/%s?flush_cache=true'%filename

http://nbviewer.ipython.org/url/portal.nersc.gov/project/openmsi/MetAtlas_005_Find_Data_in_Grouped_Files_For_Given_Atlas.ipynb?flush_cache=true
